In [1]:
import sys
sys.path.append('/media/disk1/chatgpt/zh/tabular_data')

In [2]:
from datasets import load_dataset
totto = load_dataset('../data_loader/totto_zh.py', verification_mode="no_checks", cache_dir="/media/disk2/datasets")
tabfact = load_dataset('../data_loader/tabfact.py', verification_mode="no_checks", cache_dir="/media/disk2/datasets")
sqa = load_dataset('../data_loader/sqa.py', verification_mode="no_checks", cache_dir="/media/disk2/datasets")

In [3]:
import re
def extract_yes_no_and_map(text):
    # Convert the input text to lowercase for case-insensitive matching
    text = text.lower()

    # Define regular expressions for yes/no matching
    yes_patterns = [r'\byes\b', r'\btrue\b']
    no_patterns = [r'\bno\b', r'\bfalse\b']

    # Check for "0"
    if text == "0":
        return "0"

    # Check for "1"
    if text == "1":
        return "1"

    # Check for yes
    for pattern in yes_patterns:
        if re.search(pattern, text):
            return "1"

    # Check for no
    for pattern in no_patterns:
        if re.search(pattern, text):
            return "0"

    # Return 2 if neither yes nor no is found
    return "2"

def eval_fv_match(pred_list, gold_list):
        acc = 0.0
        for pred, gold in zip(pred_list, gold_list):
            pred, gold = extract_yes_no_and_map(pred), extract_yes_no_and_map(gold)
            if pred == gold:
                acc += 1
        acc = acc / len(pred_list)
        return acc
    
def process_input(sample, instruction):
    df = pd.DataFrame(columns=sample["table"]["header"])
    for i, line in enumerate(sample['table']['rows']):
        df.loc[i] = line
    texts = [instruction,
          "the table needed to be answered: \n",
          df.to_html(),
          "query:",
          sample['statement'],
           "answer is: \n"]
    
    label = sample['label']
    return "\n".join(texts), label 

## Evaluat from local result

#### show tabfact

In [4]:
from executor import SQLManager
from sqlalchemy import create_engine
from data_loader import TableLoader, TableFormat
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine)
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
# table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
i = 1982

def show_table(data, execute=False):
    formatter = TableFormat(format='none', data=data)
    print(data['table']['id'])
    print(data['statement'])
    print(formatter.format_psql())
    # print(preds[i])
    # print(SQLs[i])
    # test_df = manager.execute_from_df(SQLs[i], formatter.all_data)
    # print(test_df)
    print(data['label'])
print(table_loader.dataset[i]['table']['caption'])
show_table(table_loader.dataset[i]) 

media in kelowna
2-18409326-1.html.csv
the call sign for astral media 's radio station be ckfr
+-------------+---------------+-----------------+-------------------------+-----------------------------------+
| frequency   | call_sign     | branding        | format                  | owner                             |
|-------------+---------------+-----------------+-------------------------+-----------------------------------|
| 1150 am     | ckfr          | am 1150         | news / talk             | astral media                      |
| 00 88.9 fm  | cbtk - fm     | cbc radio one   | public news / talk      | canadian broadcasting corporation |
| 00 89.7 fm  | cbu - fm - 3  | cbc radio 2     | public music            | canadian broadcasting corporation |
| 00 90.5 fm  | cbuf - fm - 2 | première chaîne | public news / talk      | canadian broadcasting corporation |
| 00 96.3 fm  | ckko - fm     | k96.3           | classic rock            | sun country cablevision           |
| 00 99.9

#### show wikitable

In [5]:
from executor import SQLManager
from sqlalchemy import create_engine
from data_loader import TableLoader, TableFormat
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine)
table_loader = TableLoader(table_name='wikitable', split='test', use_sample=False, small_test=False)
# table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
i = 4233



def show_table(data, execute=False):
    formatter = TableFormat(format='none', data=data)
    print(data['id'])
    print(data['query'])
    print(formatter.format_psql())
    # print(preds[i])
    # print(SQLs[i])
    # test_df = manager.execute_from_df(SQLs[i], formatter.all_data)
    # print(test_df)
    print(data['label'])

show_table(table_loader.normalize_table(table_loader.dataset[i]) )

nu-4233
how many c strings have no corresponding single glyph?
+----------------------+---------+------------+-----------+---------------------------+
| name                 | glyph   | C_string   | Unicode   | Unicode_name              |
|----------------------+---------+------------+-----------+---------------------------|
| NUL                  |         | \\0        | U+0000    | NULL (NUL)                |
| alert                |         | \\a        | U+0007    | BELL (BEL)                |
| backspace            |         | \\b        | U+0008    | BACKSPACE (BS)            |
| tab                  |         | \\t        | U+0009    | CHARACTER TABULATION (HT) |
| carriage-return      |         | \\r        | U+000D    | CARRIAGE RETURN (CR)      |
| newline              |         | \\n        | U+000A    | LINE FEED (LF)            |
| vertical-tab         |         | \\v        | U+000B    | LINE TABULATION (VT)      |
| form-feed            |         | \\f        | U+000C   

In [6]:
for ind, i in enumerate(table_loader.dataset['question']):
    print(ind, i)

0 which country had the most cyclists finish within the top 10?
1 how many people were murdered in 1940/41?
2 how long did it take for the new york americans to win the national cup after 1936?
3 alfie's birthday party aired on january 19. what was the airdate of the next episode?
4 what is the number of 1st place finishes across all events?
5 in which competition did hopley finish fist?
6 what is the total number of films with the language of kannada listed?
7 what was the number of people attending the toros mexico vs. monterrey flash game?
8 what time period had no shirt sponsor?
9 when was his first 1st place record?
10 in which three consecutive years was the record the same?
11 does pat or john have the highest total?
12 what is the combined score of year end rankings before 2009?
13 how many more ships were wrecked in lake huron than in erie?
14 what is the only character with a blank c string?
15 what was the total number of points scored by the tide in the last 3 games combine

### Evaluate Tabfact


In [7]:
import re
def extract_yes_no_and_map(text):
    # Convert the input text to lowercase for case-insensitive matching
    text = text.lower()

    # Define regular expressions for yes/no matching
    yes_patterns = [r'\byes\b', r'\btrue\b']
    no_patterns = [r'\bno\b', r'\bfalse\b']

    # Check for "0"
    if text == "0":
        return "0"

    # Check for "1"
    if text == "1":
        return "1"

    # Check for yes
    for pattern in yes_patterns:
        if re.search(pattern, text):
            return "1"

    # Check for no
    for pattern in no_patterns:
        if re.search(pattern, text):
            return "0"

    # Return 2 if neither yes nor no is found
    return "2"

def eval_fv_match(pred_list, gold_list):
        acc = 0.0
        for pred, gold in zip(pred_list, gold_list):
            pred, gold = extract_yes_no_and_map(pred), extract_yes_no_and_map(gold)
            if pred == gold:
                acc += 1
        acc = acc / len(pred_list)
        return acc
def eval_blury_string(pred_list):
    pred_label = []
    for pred in pred_list:
        predict_ans = pred.split('\n')[-1]
        if '0' in predict_ans:
            predict_ans = '0'
        elif '1' in predict_ans:
            predict_ans = '1'
        else:
            predict_ans = '2'
        pred_label.append(predict_ans)
    return pred_label

In [8]:
import pandas as pd
data = pd.read_csv('../result/answer/tabfact_test_query_gpt35_0125.csv')
data['final_answer'] = data['preds'].apply(lambda x: x.split('Answer:')[-1].strip())


In [9]:
labels = []
preds = []
ids = []
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
for i, row in data.iterrows():
    ids.append(row['ids'])
    labels.append(table_loader.normalize_table(table_loader.dataset[int(row['ids'])])['label'])
    preds.append(row['final_answer'])

In [10]:
eval_fv_match(labels, preds)

0.8194871794871795

In [11]:
acc = 0
errors = []
for i in range(len(preds)):
    pred, gold = extract_yes_no_and_map(preds[i]), extract_yes_no_and_map(labels[i])
    if pred != gold:
        print(ids[i], pred, gold)
        errors.append(i)
    else:
        acc += 1

1 0 1
3 0 1
7 1 0
11 0 1
14 0 1
16 1 0
17 1 0
21 0 1
22 0 1
26 0 1
32 0 1
37 0 1
39 1 0
42 0 1
43 1 0
46 1 0
47 1 0
51 0 1
58 1 0
59 0 1
62 0 1
63 1 0
64 1 0
66 1 0
69 0 1
72 0 1
73 1 0
76 1 0
79 0 1
80 0 1
82 0 1
84 1 0
87 1 0
91 0 1
100 0 1
101 0 1
103 1 0
106 1 0
107 1 0
108 0 1
115 2 0
121 1 0
126 1 0
133 1 0
135 1 0
138 1 0
140 0 1
148 0 1
151 0 1
160 0 1
169 0 1
180 0 1
181 1 0
184 1 0
188 0 1
192 1 0
196 1 0
199 0 1
210 0 1
211 0 1
213 1 0
216 2 0
225 1 0
227 1 0
236 1 0
240 0 1
241 0 1
249 0 1
254 1 0
259 0 1
261 0 1
265 1 0
268 0 1
276 1 0
277 1 0
281 0 1
291 0 1
292 1 0
296 1 0
300 1 0
301 0 1
309 0 1
315 0 1
341 1 0
349 1 0
350 1 0
355 0 1
362 1 0
375 0 1
390 1 0
392 1 0
393 1 0
396 1 0
397 1 0
398 1 0
408 0 1
422 1 0
425 0 1
426 1 0
430 1 0
431 1 0
432 1 0
435 0 1
436 0 1
438 0 1
439 0 1
441 1 0
442 1 0
444 1 0
457 1 0
463 1 0
473 1 0
475 0 1
476 0 1
488 0 1
489 0 1
491 1 0
493 1 0
495 0 1
499 0 1
502 1 0
503 1 0
504 0 1
510 1 0
511 1 0
517 0 1
520 0 1
522 1 0
534 1 0
538 0

### Evaluate WikiTable

In [2]:
import re
import string
import recognizers_suite
import recognizers_suite as Recognizers
from recognizers_text import Culture, ModelResult
def str_normalize(user_input, recognition_types=None):
    """A string normalizer which recognize and normalize value based on recognizers_suite"""
    user_input = str(user_input)
    user_input = user_input.replace("\\n", "; ")

    def replace_by_idx_pairs(orig_str, strs_to_replace, idx_pairs):
        assert len(strs_to_replace) == len(idx_pairs)
        last_end = 0
        to_concat = []
        for idx_pair, str_to_replace in zip(idx_pairs, strs_to_replace):
            to_concat.append(orig_str[last_end : idx_pair[0]])
            to_concat.append(str_to_replace)
            last_end = idx_pair[1]
        to_concat.append(orig_str[last_end:])
        return ''.join(to_concat)

    if recognition_types is None:
        recognition_types = [
            "datetime",
            "number",
            "ordinal",
            "percentage",
            "age",
            "currency",
            "dimension",
            "temperature",
        ]
    culture = Culture.English
    for recognition_type in recognition_types:
        if re.match("\d+/\d+", user_input):
            # avoid calculating str as 1991/92
            continue
        recognized_list = getattr(
            recognizers_suite, "recognize_{}".format(recognition_type)
        )(
            user_input, culture
        )  # may match multiple parts
        strs_to_replace = []
        idx_pairs = []
        for recognized in recognized_list:
            if not recognition_type == 'datetime':
                recognized_value = recognized.resolution['value']
                if str(recognized_value).startswith("P"):
                    # if the datetime is a period:
                    continue
                else:
                    strs_to_replace.append(recognized_value)
                    idx_pairs.append((recognized.start, recognized.end + 1))
            else:
                if recognized.resolution:  # in some cases, this variable could be none.
                    if len(recognized.resolution['values']) == 1:
                        strs_to_replace.append(
                            recognized.resolution['values'][0]['timex']
                        )  # We use timex as normalization
                        idx_pairs.append((recognized.start, recognized.end + 1))

        if len(strs_to_replace) > 0:
            user_input = replace_by_idx_pairs(user_input, strs_to_replace, idx_pairs)

    if re.match("(.*)-(.*)-(.*) 00:00:00", user_input):
        user_input = user_input[: -len("00:00:00") - 1]
        # '2008-04-13 00:00:00' -> '2008-04-13'
    return user_input


def normalize_answer(s):
    def remove_articles(text):
        return re.sub(re.compile(r"\b(a|an|the)\b", re.UNICODE), " ", text)

    def whilt_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return whilt_space_fix(remove_articles(remove_punc(lower(s))))


In [3]:

import sys, os, re, argparse
import unicodedata
from codecs import open
from math import isnan, isinf
from abc import ABCMeta, abstractmethod


################ String Normalization ################

def normalize(x):
    if not isinstance(x, str):
        x = x.decode('utf8', errors='ignore')
    # Remove diacritics
    x = ''.join(c for c in unicodedata.normalize('NFKD', x)
                if unicodedata.category(c) != 'Mn')
    # Normalize quotes and dashes
    x = re.sub(r"[‘’´`]", "'", x)
    x = re.sub(r"[“”]", "\"", x)
    x = re.sub(r"[‐‑‒–—−]", "-", x)
    while True:
        old_x = x
        # Remove citations
        x = re.sub(r"((?<!^)\[[^\]]*\]|\[\d+\]|[•♦†‡*#+])*$", "", x.strip())
        # Remove details in parenthesis
        x = re.sub(r"(?<!^)( \([^)]*\))*$", "", x.strip())
        # Remove outermost quotation mark
        x = re.sub(r'^"([^"]*)"$', r'\1', x.strip())
        if x == old_x:
            break
    # Remove final '.'
    if x and x[-1] == '.':
        x = x[:-1]
    # Collapse whitespaces and convert to lower case
    x = re.sub(r'\s+', ' ', x, flags=re.U).lower().strip()
    return x


################ Value Types ################

class Value(object):
    __metaclass__ = ABCMeta

    # Should be populated with the normalized string
    _normalized = None

    @abstractmethod
    def match(self, other):
        """Return True if the value matches the other value.
        Args:
            other (Value)
        Returns:
            a boolean
        """
        pass

    @property
    def normalized(self):
        return self._normalized


class StringValue(Value):

    def __init__(self, content):
        assert isinstance(content, str)
        self._normalized = normalize(content)
        self._hash = hash(self._normalized)

    def __eq__(self, other):
        return isinstance(other, StringValue) and self.normalized == other.normalized

    def __hash__(self):
        return self._hash

    def __str__(self):
        return 'S' + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        return self.normalized == other.normalized


class NumberValue(Value):

    def __init__(self, amount, original_string=None):
        assert isinstance(amount, (int, float))
        if abs(amount - round(amount)) < 1e-6:
            self._amount = int(amount)
        else:
            self._amount = float(amount)
        if not original_string:
            self._normalized = str(self._amount)
        else:
            self._normalized = normalize(original_string)
        self._hash = hash(self._amount)

    @property
    def amount(self):
        return self._amount

    def __eq__(self, other):
        return isinstance(other, NumberValue) and self.amount == other.amount

    def __hash__(self):
        return self._hash

    def __str__(self):
        return ('N(%f)' % self.amount) + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, NumberValue):
            return abs(self.amount - other.amount) < 1e-6
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a number.
        Return:
            the number (int or float) if successful; otherwise None.
        """
        try:
            return int(text)
        except:
            try:
                amount = float(text)
                assert not isnan(amount) and not isinf(amount)
                return amount
            except:
                return None


class DateValue(Value):

    def __init__(self, year, month, day, original_string=None):
        """Create a new DateValue. Placeholders are marked as -1."""
        assert isinstance(year, int)
        assert isinstance(month, int) and (month == -1 or 1 <= month <= 12)
        assert isinstance(day, int) and (day == -1 or 1 <= day <= 31)
        assert not (year == month == day == -1)
        self._year = year
        self._month = month
        self._day = day
        if not original_string:
            self._normalized = '{}-{}-{}'.format(
                year if year != -1 else 'xx',
                month if month != -1 else 'xx',
                day if day != '-1' else 'xx')
        else:
            self._normalized = normalize(original_string)
        self._hash = hash((self._year, self._month, self._day))

    @property
    def ymd(self):
        return (self._year, self._month, self._day)

    def __eq__(self, other):
        return isinstance(other, DateValue) and self.ymd == other.ymd

    def __hash__(self):
        return self._hash

    def __str__(self):
        return (('D(%d,%d,%d)' % (self._year, self._month, self._day))
                + str([self._normalized]))

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, DateValue):
            return self.ymd == other.ymd
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a date.
        Return:
            tuple (year, month, date) if successful; otherwise None.
        """
        try:
            ymd = text.lower().split('-')
            assert len(ymd) == 3
            year = -1 if ymd[0] in ('xx', 'xxxx') else int(ymd[0])
            month = -1 if ymd[1] == 'xx' else int(ymd[1])
            day = -1 if ymd[2] == 'xx' else int(ymd[2])
            assert not (year == month == day == -1)
            assert month == -1 or 1 <= month <= 12
            assert day == -1 or 1 <= day <= 31
            return (year, month, day)
        except:
            return None


################ Value Instantiation ################

def to_value(original_string, corenlp_value=None):
    """Convert the string to Value object.
    Args:
        original_string (basestring): Original string
        corenlp_value (basestring): Optional value returned from CoreNLP
    Returns:
        Value
    """
    if isinstance(original_string, Value):
        # Already a Value
        return original_string
    if not corenlp_value:
        corenlp_value = original_string
    # Number?
    amount = NumberValue.parse(corenlp_value)
    if amount is not None:
        return NumberValue(amount, original_string)
    # Date?
    ymd = DateValue.parse(corenlp_value)
    if ymd is not None:
        if ymd[1] == ymd[2] == -1:
            return NumberValue(ymd[0], original_string)
        else:
            return DateValue(ymd[0], ymd[1], ymd[2], original_string)
    # String.
    return StringValue(original_string)


def to_value_list(original_strings, corenlp_values=None):
    """Convert a list of strings to a list of Values
    Args:
        original_strings (list[basestring])
        corenlp_values (list[basestring or None])
    Returns:
        list[Value]
    """
    assert isinstance(original_strings, (list, tuple, set))
    if corenlp_values is not None:
        assert isinstance(corenlp_values, (list, tuple, set))
        assert len(original_strings) == len(corenlp_values)
        return list(set(to_value(x, y) for (x, y)
                        in zip(original_strings, corenlp_values)))
    else:
        return list(set(to_value(x) for x in original_strings))


################ Check the Predicted Denotations ################

def check_denotation(target_values, predicted_values):
    """Return True if the predicted denotation is correct.

    Args:
        target_values (list[Value])
        predicted_values (list[Value])
    Returns:
        bool
    """
    # Check size
    if len(target_values) != len(predicted_values):
        return False
    # Check items
    for target in target_values:
        if not any(target.match(pred) for pred in predicted_values):
            return False
    return True
def tsv_unescape(x):
    """Unescape strings in the TSV file.
    Escaped characters include:
        newline (0x10) -> backslash + n
        vertical bar (0x7C) -> backslash + p
        backslash (0x5C) -> backslash + backslash

    Args:
        x (str or unicode)
    Returns:
        a unicode
    """
    return x.replace(r'\n', '\n').replace(r'\p', '|').replace('\\\\', '\\')

def tsv_unescape_list(x):
    """Unescape a list in the TSV file.
    List items are joined with vertical bars (0x5C)

    Args:
        x (str or unicode)
    Returns:
        a list of unicodes
    """
    return [tsv_unescape(y) for y in x.split('|')]

In [4]:
import os
tagged_dataset_path = '../result/data/wikitable/tagged/data'
target_values_map = {}
for filename in os.listdir(tagged_dataset_path):
    if filename[0]=='.':
        continue
    filename = os.path.join(tagged_dataset_path, filename)
    print('Reading dataset from', filename)
    with open(filename, 'r') as fin:
        header = fin.readline().rstrip('\n').split('\t')
        for line in fin:
            stuff = dict(zip(header, line.rstrip('\n').split('\t')))
            ex_id = stuff['id']
            original_strings = tsv_unescape_list(stuff['targetValue'])
            canon_strings = tsv_unescape_list(stuff['targetCanon'])

            target_values_map[ex_id] = to_value_list(
                original_strings, canon_strings)

Reading dataset from ../result/data/wikitable/tagged/data/training.tagged
Reading dataset from ../result/data/wikitable/tagged/data/pristine-unseen-tables.tagged
Reading dataset from ../result/data/wikitable/tagged/data/pristine-seen-tables.tagged


##### read_csv

In [6]:
import pandas as pd
# data1 = pd.read_csv('./result/answer/wikitable_zh_05-03_15-27-43.csv', encoding='utf-8') 
# data2 = pd.read_csv('./result/answer/wikitable_zh_05-03_15-46-07.csv', encoding='utf-8') 
data = pd.read_csv('../result/answer/wikitable_05-14_08-17-04.csv')


In [7]:
data['final_answer'] = data['preds'].apply(lambda x: x.split('Answer:')[-1].strip())


In [21]:
acc = 0

for index, row in data.iterrows():
    id = row['ids']
    target = target_values_map[id]
    
    preds = [row['final_answer']]
    if len(target) > 1:
        pred_answer = to_value_list(row['final_answer'].split(','))
    else:
        pred_answer = to_value_list(preds)
    
    if check_denotation(pred_answer, target):
        acc += 1
    else:
        if index in [899, 1799,1439,1528,1798,2068,2248]:
            print(index,row['ids'], row['final_answer'], target)
        pass
print(acc)

1439 nu-1441 Mr B. Owen-Jones [S['mr b. owen- jones']]
1798 nu-1800 10 [N(10727.000000)['10727']]
1799 nu-1801 R Coonan [S['jeff king']]
2068 nu-2071 Former Mantoku-in Gardens (旧万徳院庭園 kyū-Mantokuin teien) [S['kikkawa motoharu fortified residence gardens']]
2248 nu-2251 Oldsmobile, Chevrolet, Toyota, Honda [S['toyota'], S['honda'], S['oldsmobile']]
2869


In [9]:
acc / len(data)

0.6624336181020549

In [10]:
len(data)

4331

In [22]:
acc = 0
for index, row in data[:100].iterrows():
    id = row['ids']
    target = target_values_map[id]
    
    preds = [row['final_answer']]
    pred_answer = to_value_list(preds)
    if check_denotation(pred_answer, target):
        acc += 1
    else:
        print(index,row['ids'], row['final_answer'], target)
        pass
print(acc)

0 nu-0 ITA [S['italy']]
2 nu-2 1953/54 [N(17.000000)['17 years']]
3 nu-3 January 20, 1995 [D(1995,1,26)['january 26, 1995']]
10 nu-10 1970, 1988, 1979 [N(2004.000000)['2004'], N(2005.000000)['2005'], N(2006.000000)['2006']]
12 nu-12 17 [N(440.000000)['440']]
13 nu-13 0 [N(7.000000)['7']]
14 nu-14 Not provided [S['space']]
15 nu-15 240 [N(68.000000)['68']]
19 nu-19 3 [N(492111.000000)['492,111']]
23 nu-23 Sayonara [S['brindabella']]
25 nu-25 8 [N(3.000000)['3']]
26 nu-26 14 [N(20.000000)['20']]
30 nu-30 Wisconsin Avenue Line [S['pennsylvania avenue metro extra line']]
34 nu-34 No other contestant(s) [S['jahaira novgorodova'], S['patricia valiahmetova'], S['mariesea mnesicu'], S['carmen jenockova'], S['anastasija larkova']]
35 nu-35 7 [N(0.000000)['0']]
37 nu-37 0 [N(10.000000)['10']]
42 nu-42 St Nikolai [S["st. mary's church"]]
44 nu-44 2014 [N(1992.000000)['1992']]
46 nu-46 12 [N(20.000000)['20']]
48 nu-48 Chile, Ecuador [S['chile'], S['ecuador']]
53 nu-53 1986 [N(1935.000000)['1935']]